<a href="https://colab.research.google.com/github/NevadaM/DCAI_ODI0524/blob/main/datacollatortesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers huggingface-hub
!pip install -q --force-reinstall 'pyarrow<15.0.0a0,>=14.0.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForCausalLM, DataCollatorForLanguageModeling
from datasets import load_dataset


In [4]:
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = TFAutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [5]:
text_string = "Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism). Humans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers' struggles for emancipation. Various anarchist schools of thought formed during this period. Anarchists have taken part in several revolutions, most notably in the Paris Commune, the Russian Civil War and the Spanish Civil War, whose end marked the end of the classical era of anarchism."
len(text_string)

1361

In [6]:
text_tokenized = tokenizer.encode(text_string, return_tensors="np")

In [7]:
from collections import deque

In [8]:
def chunk(arr, feature_len=16, padding_token=1, return_tf_datasets=False):
  features = []
  chunk_deque = deque([padding_token for _ in range(feature_len)], maxlen=feature_len)

  for i in arr:
    chunk_deque.append(i)
    features.append(list(chunk_deque))

  if return_tf_datasets:
    return tf.data.Dataset.from_tensor_slices(features).batch(4)

  return features

f, l = chunk(text_tokenized[-1], return_tf_datasets=True)
# f, l
# f, l = chunk(text_tokenized[-1])


In [9]:
def train_on_batch_datasets(model, features, epochs=1):
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.005) ## this is high for testing purposes! change
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  for epoch in range(epochs):
    counter = 0
    for batch_features in features:
      with tf.GradientTape() as tape:
        loss_value = loss(features, model(batch_features)['logits'])
        # print(loss_value)

      grads = tape.gradient(loss_value, model.trainable_variables)
      # print(grads)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      counter += 1
      if counter % 5 == 0:
        print(f'Step: {counter}   Loss: {loss_value}')

  return model



In [39]:
while True:
  try:
    model = TFAutoModelForCausalLM.from_pretrained(model_name)
    break
  except:
    continue

All model checkpoint layers were used when initializing TFOPTForCausalLM.

All the layers of TFOPTForCausalLM were initialized from the model checkpoint at facebook/opt-125m.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOPTForCausalLM for predictions without further training.


In [12]:
## benchmarking
text_prompt = "Anarchism is a political philosophy and movement that is skeptical"
text_tokenized = tokenizer.encode(text_prompt, return_tensors="tf")
output = model.generate(text_tokenized, do_sample=True, max_length=100)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

output_text

'Anarchism is a political philosophy and movement that is skeptical of the authority of the church in its own way and thus seeks to establish an ideology that embraces the concept of man as its leader and should be understood as not only about what is right, but also how it has managed to become the most effective political organization in the United States.\n\nIn 1843, the first American Civil War was fought in the South. In the aftermath, the American Revolution was also fought in the South because'

In [14]:
## training
model = train_on_batch_datasets(model, f, l, epochs=25)

Step: 5   Loss: 4.597414016723633
Step: 10   Loss: 4.135599136352539
Step: 15   Loss: 5.197532653808594
Step: 20   Loss: 4.825636863708496
Step: 25   Loss: 6.162888526916504
Step: 30   Loss: 5.2282562255859375
Step: 35   Loss: 5.071022987365723
Step: 40   Loss: 4.711303234100342
Step: 45   Loss: 4.263737201690674
Step: 50   Loss: 5.433650970458984
Step: 55   Loss: 5.973588943481445
Step: 60   Loss: 5.200301170349121
Step: 5   Loss: 4.386838912963867
Step: 10   Loss: 4.193190097808838
Step: 15   Loss: 5.619240760803223
Step: 20   Loss: 4.913490295410156
Step: 25   Loss: 5.725854873657227
Step: 30   Loss: 4.705273628234863
Step: 35   Loss: 4.174762725830078
Step: 40   Loss: 3.961120128631592
Step: 45   Loss: 3.8025498390197754
Step: 50   Loss: 4.760210037231445
Step: 55   Loss: 5.013979434967041
Step: 60   Loss: 4.495548725128174
Step: 5   Loss: 4.419894218444824
Step: 10   Loss: 4.55423641204834
Step: 15   Loss: 5.069760799407959
Step: 20   Loss: 4.72958517074585
Step: 25   Loss: 6.4551

In [15]:
## testing
text_prompt = "Anarchism is a political philosophy and movement that is "
text_tokenized = tokenizer.encode(text_prompt, return_tensors="tf")
output = model.generate(text_tokenized, do_sample=True, max_length=100)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

output_text

"Anarchism is a political philosophy and movement that is  movement of several this revolutions the of. formed world for most the this the the in significant anarchism several authorityists the movement Anarch the the the Anarch had anarchist have movement notably in the emancipation for in, in anarchism mostth anarchist several. revolutionsists, the in the most the this for,, part of all in of in authority, Comm Paris most century world all taken. anarchism role have end the, Paris' world of,."

In [46]:
tokenizer.encode('marked')

[2, 18584]

in the logits is it constantly predicting beginning of sentence tokens?

In [40]:
x = tokenizer.encode("Anarchism is a political philosophy and movement that is sceptical of all just", return_tensors='tf')[:16]
y = tokenizer.encode("Anarchism is a political philosophy and movement that is sceptical of all just")[-1]

preds = model.predict(x)

1/1 [==============================] - 12s 12s/step


In [41]:
preds['logits'].shape

(1, 17, 50272)

In [49]:
x_tokens = [tokenizer.decode(x[0][i]) for i in range(17)]
preds_text = [tokenizer.decode(np.argmax(preds['logits'][0][i])) for i in range(17)]

In [52]:
loss_calc = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [53]:
loss_calc(x, preds['logits'])

<tf.Tensor: shape=(), dtype=float32, numpy=8.44039>

In [51]:
x_tokens

['</s>',
 'An',
 'arch',
 'ism',
 ' is',
 ' a',
 ' political',
 ' philosophy',
 ' and',
 ' movement',
 ' that',
 ' is',
 ' scept',
 'ical',
 ' of',
 ' all',
 ' just']

In [44]:
preds_text

['I',
 'ime',
 'o',
 ' is',
 ' a',
 ' form',
 ' ideology',
 ' that',
 ' philosophy',
 ' that',
 ' seeks',
 ' based',
 'ical',
 ' of',
 ' the',
 ' forms',
 'ifications']

In [45]:
x

<tf.Tensor: shape=(1, 17), dtype=int32, numpy=
array([[    2,  4688, 13161,  1809,    16,    10,   559, 10561,     8,
         2079,    14,    16, 22865,  3569,     9,    70,    95]],
      dtype=int32)>